In [ ]:
import sys
get_ipython().system('{sys.executable} -m pip install keras')
get_ipython().system('{sys.executable} -m pip install pillow')
get_ipython().system('{sys.executable} -m pip install tensorflow')

In [12]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [13]:
batch_size = 128
num_classes = 10
epochs = 10

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [14]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
class MNIST_CNN(object):
    def __init__(self):
        self.load_mnist()

        # prepare pixel data
        self.trainX, self.testX = self.prep_pixels(self.trainX, self.testX)

    def load_mnist(self):
        """
        load train and test dataset
        """
        # load dataset
        (self.trainX, self.trainY), (self.testX, self.testY0) = mnist.load_data()

        # reshape dataset to have a single channel
        self.trainX = self.trainX.reshape((self.trainX.shape[0], 28, 28, 1))

        self.testX = self.testX.reshape((self.testX.shape[0], 28, 28, 1))

        # one hot encode target values
        self.trainY = to_categorical(self.trainY)
        self.testY = to_categorical(self.testY0)

    def prep_pixels(self, train_set, test_set):
        """
        Scale pixels
        :param train_set:
        :param test_set:
        :return:
        """
        # convert from integers to floats
        train_norm = train_set.astype('float32')
        test_norm = test_set.astype('float32')

        # normalize to range 0-1
        train_norm = train_norm / 255.0
        test_norm = test_norm / 255.0

        # return normalized images
        return train_norm, test_norm

    def define_model(self):
        """
        define cnn model
        :return:
        """
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
        model.add(BatchNormalization())     # Improvement to Learning
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
        model.add(BatchNormalization())     # Improvement to Learning
        model.add(Dense(10, activation='softmax'))

        # compile model
        opt = SGD(lr=0.01, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
        print("The model has successfully trained")
        model.save('mnist.h5')
        print("Saving the model as mnist.h5")
        return model

    def train(self, epochs=10):
        # define model
        model = self.define_model()

        # fit model
        history = model.fit(self.trainX, self.trainY,
                            epochs=epochs,
                            batch_size=32,
                            validation_data=(self.testX, self.testY),
                            verbose=0)

        # evaluate model
        loss, acc = model.evaluate(self.testX, self.testY, verbose=0)
        print('> %.4e %.3f' % (loss, acc * 100.0))

        #y_prob = model.predict(self.testX, batch_size=1)
        #print(y_prob)

        y_pred = model.predict_classes(self.testX, batch_size=1)
        #print(y_pred)

        #print(self.testY0)
        self.performance(self.testY0, y_pred)

    def performance(self, y_test, y_predict):
        from sklearn.metrics import classification_report, confusion_matrix
        import pandas as pd

        report = classification_report(y_test, y_predict)
        print(report)


def test():
    # CNN
    prj = MNIST_CNN()
    prj.train(epochs=5)   


if __name__ == '__main__':
    test()

    exit(0)

Epoch 1/10
469/469 [==============================] - 56s 119ms/step - loss: 0.2984 - accuracy: 0.9135 - val_loss: 0.0961 - val_accuracy: 0.9762
Epoch 2/10
469/469 [==============================] - 56s 118ms/step - loss: 0.0579 - accuracy: 0.9843 - val_loss: 0.0544 - val_accuracy: 0.9841
Epoch 3/10
469/469 [==============================] - 53s 114ms/step - loss: 0.0340 - accuracy: 0.9913 - val_loss: 0.0460 - val_accuracy: 0.9848
Epoch 4/10
469/469 [==============================] - 50s 108ms/step - loss: 0.0224 - accuracy: 0.9946 - val_loss: 0.0431 - val_accuracy: 0.9866
Epoch 5/10
469/469 [==============================] - 52s 110ms/step - loss: 0.0158 - accuracy: 0.9967 - val_loss: 0.0439 - val_accuracy: 0.9859
Epoch 6/10
469/469 [==============================] - 52s 112ms/step - loss: 0.0109 - accuracy: 0.9982 - val_loss: 0.0412 - val_accuracy: 0.9871
Epoch 7/10
469/469 [==============================] - 52s 110ms/step - loss: 0.0077 - accuracy: 0.9990 - val_loss: 0.0419 - val_ac

C:\Users\justina32\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.98      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.98      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [11]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
model = load_model('prj_cnn.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()